In [43]:
from pymongo import MongoClient
from datetime import datetime

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()
    

Data inserted successfully


**[문제 1: 특정 장르의 책 찾기]**

- **문제 설명**:
데이터베이스에 새로운 필드로 **`genre`**를 책 데이터에 추가하였습니다. 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [42]:
def find_books_by_genre(db, genre):
    books_collection = db.books
    query = {"genre": genre}
    projection = {"_id": 0, "title": 1, "author": 1}

    books = books_collection.find(query, projection)
    for book in books:
        print(book)

#함수 실행 코드
with MongoClient("mongodb://localhost:27017/") as client:
    db = client.local
    find_books_by_genre(db, "fantasy")



{'title': 'The Hobbit', 'author': 'J.R.R. Tolkien'}
{'title': 'Harry Potter', 'author': 'J.K. Rowling'}


**[문제 2: 감독별 평균 영화 평점 계산]**

- **문제 설명**:
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**: 
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [41]:
def calculate_average_rating_by_director():
    with MongoClient("mongodb://localhost:27017/") as client:
        db = client.local
        pipeline = [
            {"$group": {"_id": "$director","average_rating": {"$avg": "$rating"}}},
            {"$sort": {"average_rating": -1}}]

        results = db.movies.aggregate(pipeline)

        print("감독별 평균 영화 평점")
        for director in results:
            print(f"- {director['_id']}: {director['average_rating']:.2f}")
            
calculate_average_rating_by_director()


감독별 평균 영화 평점
- Christopher Nolan: 8.80


**[문제 3: 특정 사용자의 최근 행동 조회]**

- **문제 설명**:
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [40]:
def get_recent_actions_by_user(user_id):
    with MongoClient("mongodb://localhost:27017/") as client:
        db = client.local
        actions = db.user_actions.find(
            {"user_id": user_id}
        ).sort("timestamp", -1).limit(5)

        print(f"🕒 사용자 {user_id}의 최근 행동 5개:")
        for action in actions:
            print(f"- {action['action']} at {action['timestamp']}")

get_recent_actions_by_user(1)

🕒 사용자 1의 최근 행동 5개:
- view at 2023-04-12 09:00:00
- view at 2023-04-12 09:00:00
- view at 2023-04-12 09:00:00
- view at 2023-04-12 09:00:00
- view at 2023-04-12 09:00:00


**[문제 4: 출판 연도별 책의 수 계산]**

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [47]:
from pymongo import MongoClient

def count_books_by_year():
    with MongoClient("mongodb://localhost:27017/") as client:
        db = client.local
        pipeline = [
            {"$group": {"_id": "$year","count": { "$sum": 1 }}},{"$sort": { "count": -1 }}]

        results = db.books.aggregate(pipeline)

        print("출판 연도별 책 수")
        for row in results:
            print(f"- {row['_id']}년: {row['count']}권")
            
count_books_by_year()

출판 연도별 책 수
- 2002년: 3권
- 2009년: 2권
- 1987년: 2권
- 1937년: 1권
- 1997년: 1권


**[문제 5: 특정 사용자의 행동 유형 업데이트]**

- **문제 설명**:
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [49]:
def update_user_actions_before_date():
    with MongoClient("mongodb://localhost:27017/") as client:
        db = client.local
        result = db.user_actions.update_many(
            {"user_id": 1,"action": "view","timestamp": {"$lt": datetime(2023, 4, 13)}},{"$set": {"action": "seen"}})
        print(f"업데이트된 문서 수: {result.modified_count}")

update_user_actions_before_date()



업데이트된 문서 수: 7
